In [2]:
Table customers {
    customer_id integer [primary key] ## Уникальный идентификатор клиента
    first_name varchar ## Имя клиента
    last_name varchar ## Фамилия клиента
    gender varchar ## Пол
    DOB date ## Дата рождения
    job_title varchar ## Должность
    job_industry_category varchar ## Категория индустрии
    wealth_segment varchar ## Сегмент благосостояния
    deceased_indicator varchar ## Признак умершего клиента
    owns_car varchar ## Владеет ли клиент автомобилем (Yes/No)
    address varchar ## Адрес
    postcode integer ## Почтовый индекс
    state varchar ## Штат
    country varchar ## Страна
    property_valuation integer ## Оценочная стоимость собственности
}

Table products {
    product_id integer [primary key] ## Уникальный идентификатор продукта
    brand varchar ## Бренд
    product_line varchar ## Линейка продукта
    product_class varchar ## Класс продукта
    product_size varchar ## Размер продукта
    list_price decimal ## Цена продукта
    standard_cost decimal ## Себестоимость продукта
}

Table transactions {
    transaction_id integer [primary key] ## Уникальный идентификатор транзакции
    customer_id integer [ref: > customers.customer_id] ## Ссылка на клиента
    product_id integer [ref: > products.product_id] ## Ссылка на продукт
    transaction_date timestamp ## Дата транзакции
    online_order boolean ## Оформлен ли заказ онлайн
    order_status varchar ## Статус заказа
}

SyntaxError: invalid syntax (4117166252.py, line 1)

![Структура БД](Screenshot_1.png)



## 1. Перевод в Первую нормальную форму (1НФ)

- В некоторых колонках могут встречаться многозначные значения (например, если один клиент может иметь несколько адресов или несколько сегментов богатства).
- Поля job_title и job_industry_category зависят от клиента, но могут повторяться.

![Описание изображения](Screenshot_2.png)



## 2. Перевод во Вторую нормальную форму (2НФ)

- wealth_segment и property_valuation зависят только от customer_id, но не связаны с транзакциями.

- job_title и job_industry_category относятся к клиенту и должны быть отделены от customers.

![Описание изображения](Screenshot_3.png)



## 3. Перевод в Третью нормальную форму (3НФ)

- postcode определяет state и country, но они находятся в addresses, а значит, зависят от postcode.

- wealth_segment и property_valuation связаны между собой (определенный wealth_segment всегда имеет типичный property_valuation).

![Описание изображения](Screenshot_4.png)

In [5]:
import sqlite3

# Подключение к базе данных (замени на свою БД)
conn = sqlite3.connect("database.db")  
cursor = conn.cursor()

# Создание таблицы клиентов
cursor.execute("""
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    gender TEXT CHECK(gender IN ('Male', 'Female')),
    dob DATE NOT NULL,
    job_title TEXT,
    job_industry_category TEXT,
    wealth_segment TEXT,
    owns_car TEXT CHECK(owns_car IN ('Yes', 'No')),
    address TEXT,
    postcode INTEGER,
    state TEXT,
    country TEXT,
    property_valuation INTEGER
);
""")

# Создание таблицы транзакций
cursor.execute("""
CREATE TABLE IF NOT EXISTS transactions (
    transaction_id INTEGER PRIMARY KEY,
    product_id INTEGER NOT NULL,
    customer_id INTEGER NOT NULL,
    transaction_date DATE NOT NULL,
    online_order TEXT CHECK(online_order IN ('True', 'False')),
    order_status TEXT,
    brand TEXT,
    product_line TEXT,
    product_class TEXT,
    product_size TEXT,
    list_price REAL NOT NULL,
    standard_cost REAL NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
""")

# Создание таблицы продуктов (опционально, если есть отдельный справочник)
cursor.execute("""
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY,
    brand TEXT NOT NULL,
    product_line TEXT,
    product_class TEXT,
    product_size TEXT
);
""")

conn.commit()
print("Таблицы успешно созданы!")


Таблицы успешно созданы!


In [9]:
import pandas as pd

# Загрузка данных
customers_df = pd.read_excel(r"C:\Users\Home\Downloads\customer_and_transaction.xlsx", sheet_name="customer")
transactions_df = pd.read_excel(r"C:\Users\Home\Downloads\customer_and_transaction.xlsx", sheet_name="transaction")

# Запись данных в таблицы customers
customers_df.to_sql("customer", conn, if_exists="append", index=False)

# Запись данных в таблицы transactions
transactions_df.to_sql("transaction", conn, if_exists="append", index=False)

print("Данные успешно загружены в базу данных!")


Данные успешно загружены в базу данных!


In [10]:
print(pd.read_sql("SELECT * FROM customers LIMIT 5;", conn))
print(pd.read_sql("SELECT * FROM transactions LIMIT 5;", conn))

Empty DataFrame
Columns: [customer_id, first_name, last_name, gender, dob, job_title, job_industry_category, wealth_segment, owns_car, address, postcode, state, country, property_valuation]
Index: []
Empty DataFrame
Columns: [transaction_id, product_id, customer_id, transaction_date, online_order, order_status, brand, product_line, product_class, product_size, list_price, standard_cost]
Index: []
